# Master Intersections and Collisions with attributes

The following code takes in CSV with the master intersection and r value data, and merges the dataset provided by the Toronto Data Portal that classifies each intersection.

#### Import packages and libraries

In [1]:
import pandas as pd
import seaborn as sns
from datetime import datetime
import numpy as np
import requests
from shapely.geometry import Point
import re
import geopandas as gpd

# 1.1 Import Master Intersection CSV

In [2]:
intersection_r = pd.read_csv('intersection_r.csv')
intersection_r.head()

,count_location,ROAD_CLASS,TRAFFCTL,num_of_collisions,total_traffic,average_int_width,years,r
0,BATHURST ST AT COLLEGE ST (PX 300),Major Arterial,No Control,18,20083.944444,0.015,11,1.488150
1,JANE ST AT ST JOHNS RD (PX 523),Major Arterial,Traffic Signal,3,11620.000000,0.015,11,0.428685
2,CHURCH ST AT GOULD ST (PX 993),Minor Arterial,Traffic Signal,10,9440.000000,0.015,11,1.758941
3,CHARLES ST E AT CHURCH ST (PX 225),Minor Arterial,No Control,2,11585.000000,0.015,11,0.286653
4,KING ST E AT PRINCESS ST,Major Arterial,No Control,11,5304.000000,0.015,11,3.443597


In [3]:
intersection_r.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1701 entries, 0 to 1700
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   count_location     1701 non-null   object 
 1   ROAD_CLASS         1701 non-null   object 
 2   TRAFFCTL           1701 non-null   object 
 3   num_of_collisions  1701 non-null   int64  
 4   total_traffic      1701 non-null   float64
 5   average_int_width  1701 non-null   float64
 6   years              1701 non-null   int64  
 7   r                  1701 non-null   float64
dtypes: float64(3), int64(2), object(3)
memory usage: 106.4+ KB


# 1.2 Import Intersection attributes dataset 

In [4]:
inter_att = (requests.get('https://ckan0.cf.opendata.inter.prod-toronto.ca/dataset/2c83f641-7808-49ba-b80f-7011851d4e27/resource/8e825e33-d7e1-4e59-b247-5868bf7d66a9/download/Centreline%20Intersection.geojson').
            json()
                   )

In [5]:
inter_att

{'type': 'FeatureCollection',
 'crs': {'type': 'name',
  'properties': {'name': 'urn:ogc:def:crs:OGC:1.3:CRS84'}},
 'features': [{'type': 'Feature',
   'properties': {'_id': 1,
    'INTERSECTION_ID': 13470264,
    'DATE_EFFECTIVE': '',
    'DATE_EXPIRY': '',
    'ELEVATION_ID': 13,
    'INTERSECTION_DESC': 'Robindale Ave / Rimilton Ave',
    'CLASSIFICATION': 'MNRSL',
    'CLASSIFICATION_DESC': 'Minor-Single Level',
    'NUMBER_OF_ELEVATIONS': 1,
    'ELEVATION_FEATURE_CODE': 501300,
    'ELEVATION_FEATURE_CODE_DESC': 'Minor',
    'ELEVATION_LEVEL': 0,
    'ELEVATION': 0.0,
    'ELEVATION_UNIT': '',
    'HEIGHT_RESTRICTION': 0.0,
    'HEIGHT_RESTRICTION_UNIT': '',
    'STATE': 8,
    'TRANS_ID_CREATE': 200000,
    'TRANS_ID_EXPIRE': -1,
    'OBJECTID': 1},
   'geometry': {'type': 'MultiPoint',
    'coordinates': [[-79.5310702158097, 43.6072425849711]]}},
  {'type': 'Feature',
   'properties': {'_id': 2,
    'INTERSECTION_ID': 13470193,
    'DATE_EFFECTIVE': '',
    'DATE_EXPIRY': '',
 

In [6]:
inter_att.keys()

dict_keys(['type', 'crs', 'features'])

In [7]:
# Initialize an empty list to store flattened dictionaries
inter_att_df = []

# Iterate through the 'features' list in the JSON intersection attributes data
for feature in inter_att['features']:
    # properties entail the characteristics of the crash
    properties = feature['properties']
    # geometry entails the location
    geometry = feature['geometry']
    flat_properties = properties.copy()
    flat_properties.update(geometry)
    inter_att_df.append(flat_properties)
    
# Create a motor_collisions DataFrame from the flattened inter_att_df
inter_att_df = pd.DataFrame(inter_att_df)
inter_att_df.head()

,_id,INTERSECTION_ID,DATE_EFFECTIVE,DATE_EXPIRY,ELEVATION_ID,INTERSECTION_DESC,CLASSIFICATION,CLASSIFICATION_DESC,NUMBER_OF_ELEVATIONS,ELEVATION_FEATURE_CODE,...,ELEVATION,ELEVATION_UNIT,HEIGHT_RESTRICTION,HEIGHT_RESTRICTION_UNIT,STATE,TRANS_ID_CREATE,TRANS_ID_EXPIRE,OBJECTID,type,coordinates
0,1,13470264,,,13,Robindale Ave / Rimilton Ave,MNRSL,Minor-Single Level,1,501300,...,0.0,,0.0,,8,200000,-1,1,MultiPoint,"[[-79.5310702158097, 43.6072425849711]]"
1,2,13470193,,,4718,Bellman Ave / Valermo Dr,MNRSL,Minor-Single Level,1,501300,...,0.0,,0.0,,8,200000,-1,4,MultiPoint,"[[-79.5313732423075, 43.609600012102]]"
2,3,13470188,,,32728,Rimilton Ave / Valermo Dr,SEUSL,Pseudo Intersection-Single Level,1,509200,...,0.0,,0.0,,8,200000,-1,5,MultiPoint,"[[-79.5301175801351, 43.6098292200395]]"
3,4,13470203,,,21669,Valermo Dr / Goa Crt,MNRSL,Minor-Single Level,1,501300,...,0.0,,0.0,,8,200000,-1,7,MultiPoint,"[[-79.5331747278101, 43.6091899836547]]"
4,5,13470228,,,36820,Valermo Dr / Thirtieth St,MNRSL,Minor-Single Level,1,501300,...,0.0,,0.0,,8,200000,-1,9,MultiPoint,"[[-79.5355925204638, 43.6086391702897]]"


In [8]:
inter_att_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55149 entries, 0 to 55148
Data columns (total 22 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   _id                          55149 non-null  int64  
 1   INTERSECTION_ID              55149 non-null  int64  
 2   DATE_EFFECTIVE               55149 non-null  object 
 3   DATE_EXPIRY                  55149 non-null  object 
 4   ELEVATION_ID                 55149 non-null  int64  
 5   INTERSECTION_DESC            55149 non-null  object 
 6   CLASSIFICATION               55149 non-null  object 
 7   CLASSIFICATION_DESC          55149 non-null  object 
 8   NUMBER_OF_ELEVATIONS         55149 non-null  int64  
 9   ELEVATION_FEATURE_CODE       55149 non-null  int64  
 10  ELEVATION_FEATURE_CODE_DESC  55149 non-null  object 
 11  ELEVATION_LEVEL              55149 non-null  int64  
 12  ELEVATION                    55149 non-null  float64
 13  ELEVATION_UNIT  

# 2.1 Grab geometry from master dataframe

In [9]:
# Import master_df thus far
master_df = pd.read_csv('master_gdf.csv')
master_df.head()

C:\Users\ianto\Python\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (0,47,49) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,ACCNUM,YEAR,DATE,TIME,collision_datetime,STREET1,STREET2,OFFSET,ROAD_CLASS,DISTRICT,...,hourly_ex_peds,hourly_wx_peds,hourly_nx_bike,hourly_sx_bike,hourly_ex_bike,hourly_wx_bike,hourly_nx_other,hourly_sx_other,hourly_ex_other,hourly_wx_other
0,1237939,2011.0,2011-03-09,726,2011-03-09 07:26:00,COLLEGE Stre,BATHURST Stre,None,Major Arterial,Toronto and East York,...,208.0,160.0,50.0,2.0,43.0,312.0,0.0,0.0,0.0,0.0
1,1232522,2011.0,2011-03-09,1145,2011-03-09 11:45:00,JANE ST,ST JOHNS RD,None,Major Arterial,Toronto and East York,...,59.0,36.0,0.0,2.0,5.0,0.0,0.0,0.0,0.0,0.0
2,1232522,2011.0,2011-03-09,1145,2011-03-09 11:45:00,JANE ST,ST JOHNS RD,None,Major Arterial,Toronto and East York,...,59.0,36.0,0.0,2.0,5.0,0.0,0.0,0.0,0.0,0.0
3,1232522,2011.0,2011-03-09,1145,2011-03-09 11:45:00,JANE ST,ST JOHNS RD,None,Major Arterial,Toronto and East York,...,59.0,36.0,0.0,2.0,5.0,0.0,0.0,0.0,0.0,0.0
4,1222866,2011.0,2011-03-12,1917,2011-03-12 19:17:00,CHURCH ST,GOULD ST,None,Minor Arterial,Toronto and East York,...,222.0,97.0,106.0,27.0,31.0,9.0,0.0,0.0,0.0,0.0


In [10]:
# Assuming 'geometry' is the column containing POINT strings
pattern = re.compile(r'POINT \((-?\d+\.\d+) (-?\d+\.\d+)\)')

def extract_coordinates(text):
    match = pattern.match(text)
    if match:
        return Point(float(match.group(1)), float(match.group(2)))
    else:
        return None

# Apply the function to create Point geometries
master_df['geometry'] = master_df['count_geometry'].apply(extract_coordinates)

# Convert master_df to a geodataframe
master_gdf = gpd.GeoDataFrame(master_df, geometry='geometry')
master_gdf.head()

,ACCNUM,YEAR,DATE,TIME,collision_datetime,STREET1,STREET2,OFFSET,ROAD_CLASS,DISTRICT,...,hourly_wx_peds,hourly_nx_bike,hourly_sx_bike,hourly_ex_bike,hourly_wx_bike,hourly_nx_other,hourly_sx_other,hourly_ex_other,hourly_wx_other,geometry
0,1237939,2011.0,2011-03-09,726,2011-03-09 07:26:00,COLLEGE Stre,BATHURST Stre,None,Major Arterial,Toronto and East York,...,160.0,50.0,2.0,43.0,312.0,0.0,0.0,0.0,0.0,POINT (-79.40772 43.65648)
1,1232522,2011.0,2011-03-09,1145,2011-03-09 11:45:00,JANE ST,ST JOHNS RD,None,Major Arterial,Toronto and East York,...,36.0,0.0,2.0,5.0,0.0,0.0,0.0,0.0,0.0,POINT (-79.48967 43.66199)
2,1232522,2011.0,2011-03-09,1145,2011-03-09 11:45:00,JANE ST,ST JOHNS RD,None,Major Arterial,Toronto and East York,...,36.0,0.0,2.0,5.0,0.0,0.0,0.0,0.0,0.0,POINT (-79.48967 43.66199)
3,1232522,2011.0,2011-03-09,1145,2011-03-09 11:45:00,JANE ST,ST JOHNS RD,None,Major Arterial,Toronto and East York,...,36.0,0.0,2.0,5.0,0.0,0.0,0.0,0.0,0.0,POINT (-79.48967 43.66199)
4,1222866,2011.0,2011-03-12,1917,2011-03-12 19:17:00,CHURCH ST,GOULD ST,None,Minor Arterial,Toronto and East York,...,97.0,106.0,27.0,31.0,9.0,0.0,0.0,0.0,0.0,POINT (-79.37785 43.65821)


In [11]:
# Assign coordinate system
master_gdf.crs = 'epsg:26917'

In [12]:
print(master_gdf.crs)

epsg:26917


# 2.2 Assignm geometry to intersection attribute dataset

In [13]:
# Use points from and lambda to extract coordinates and convert df to gdf.
intersection_gdf = gpd.GeoDataFrame(inter_att_df, 
                                    geometry = gpd.points_from_xy(inter_att_df['coordinates'].
                                                                apply(lambda x : x[0][0]), 
                                                                inter_att_df['coordinates'].
                                                                apply(lambda x : x[0][1])
                                                                 )
                                   )
intersection_gdf.head()

,_id,INTERSECTION_ID,DATE_EFFECTIVE,DATE_EXPIRY,ELEVATION_ID,INTERSECTION_DESC,CLASSIFICATION,CLASSIFICATION_DESC,NUMBER_OF_ELEVATIONS,ELEVATION_FEATURE_CODE,...,ELEVATION_UNIT,HEIGHT_RESTRICTION,HEIGHT_RESTRICTION_UNIT,STATE,TRANS_ID_CREATE,TRANS_ID_EXPIRE,OBJECTID,type,coordinates,geometry
0,1,13470264,,,13,Robindale Ave / Rimilton Ave,MNRSL,Minor-Single Level,1,501300,...,,0.0,,8,200000,-1,1,MultiPoint,"[[-79.5310702158097, 43.6072425849711]]",POINT (-79.53107 43.60724)
1,2,13470193,,,4718,Bellman Ave / Valermo Dr,MNRSL,Minor-Single Level,1,501300,...,,0.0,,8,200000,-1,4,MultiPoint,"[[-79.5313732423075, 43.609600012102]]",POINT (-79.53137 43.60960)
2,3,13470188,,,32728,Rimilton Ave / Valermo Dr,SEUSL,Pseudo Intersection-Single Level,1,509200,...,,0.0,,8,200000,-1,5,MultiPoint,"[[-79.5301175801351, 43.6098292200395]]",POINT (-79.53012 43.60983)
3,4,13470203,,,21669,Valermo Dr / Goa Crt,MNRSL,Minor-Single Level,1,501300,...,,0.0,,8,200000,-1,7,MultiPoint,"[[-79.5331747278101, 43.6091899836547]]",POINT (-79.53317 43.60919)
4,5,13470228,,,36820,Valermo Dr / Thirtieth St,MNRSL,Minor-Single Level,1,501300,...,,0.0,,8,200000,-1,9,MultiPoint,"[[-79.5355925204638, 43.6086391702897]]",POINT (-79.53559 43.60864)


In [14]:
# Like master_gdf, assign coordinate system
intersection_gdf.crs = 'epsg:26917'
print(intersection_gdf.crs)

epsg:26917


# 3.1 Merge intersection_gdf to master_gdf

In [15]:
# Perform merge below
master_att_gdf = gpd.sjoin(master_gdf, intersection_gdf, how='inner', op='intersects')
master_att_gdf.head()

C:\Users\ianto\Python\lib\site-packages\IPython\core\interactiveshell.py:3357: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if (await self.run_code(code, result,  async_=asy)):


,ACCNUM,YEAR,DATE,TIME,collision_datetime,STREET1,STREET2,OFFSET,ROAD_CLASS,DISTRICT,...,ELEVATION,ELEVATION_UNIT,HEIGHT_RESTRICTION,HEIGHT_RESTRICTION_UNIT,STATE,TRANS_ID_CREATE,TRANS_ID_EXPIRE,OBJECTID,type,coordinates
8,1233009,2011.0,2011-03-15,852,2011-03-15 08:52:00,KING Stre E,PRINCESS Stre,None,Major Arterial,Toronto and East York,...,0.0,,0.0,,8,200000,-1,29443,MultiPoint,"[[-79.3670506171421, 43.6515475861724]]"
9,1233009,2011.0,2011-03-15,852,2011-03-15 08:52:00,KING Stre E,PRINCESS Stre,None,Major Arterial,Toronto and East York,...,0.0,,0.0,,8,200000,-1,29443,MultiPoint,"[[-79.3670506171421, 43.6515475861724]]"
10,1233009,2011.0,2011-03-15,852,2011-03-15 08:52:00,KING Stre E,PRINCESS Stre,None,Major Arterial,Toronto and East York,...,0.0,,0.0,,8,200000,-1,29443,MultiPoint,"[[-79.3670506171421, 43.6515475861724]]"
11,1233009,2011.0,2011-03-15,852,2011-03-15 08:52:00,KING Stre E,PRINCESS Stre,None,Major Arterial,Toronto and East York,...,0.0,,0.0,,8,200000,-1,29443,MultiPoint,"[[-79.3670506171421, 43.6515475861724]]"
12,1233009,2011.0,2011-03-15,852,2011-03-15 08:52:00,KING Stre E,PRINCESS Stre,None,Major Arterial,Toronto and East York,...,0.0,,0.0,,8,200000,-1,29443,MultiPoint,"[[-79.3670506171421, 43.6515475861724]]"


In [16]:
# Save file
master_att_gdf.to_csv("master_att_gdf.csv", index=False)